In [9]:
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

In [10]:
import numpy as np
from dataset.mnist import load_mnist
from collections import OrderedDict
# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

In [11]:
class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self,x):
        self.x = x
        out = np.dot(x,self.W) + self.b
        return out
    def backward(self,dout):
        dx = np.dot(dout,self.W.T)
        self.dW = np.dot(self.x.T,dout)
        self.db = np.sum(dout,axis=0)
        return dx

In [12]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
    def forward(self,x,t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self.t)
        return self.loss
    def backward(self,dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

In [13]:
class Sigmoid:
    def __init__(self):
        self.out = None
    def forward(self,x):
        out = 1/(1+np.exp(-x))
        self.out = out
        return out
    def backward(self,dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

In [14]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

In [15]:
class TwoLayerNeuralNet():
    def __init__(self,input_size=3072,hidden_size=20,output_size=10,weight_init_std=0.01):
        self.params = {}
        self.params["W1"] = np.random.randn(input_size,hidden_size) * weight_init_std
        self.params["B1"] = np.zeros(hidden_size)
        self.params["W2"] = np.random.randn(hidden_size,output_size) * weight_init_std
        self.params["B2"] = np.zeros(output_size)
        self.layers = OrderedDict()
        self.layers["Affine1"] =  Affine(self.params['W1'], self.params['B1'])
        self.layers["Relu1"] = Relu()
        self.layers["Affine2"] =  Affine(self.params['W2'], self.params['B2'])
        self.lastLayer = SoftmaxWithLoss()
    def predict(self,x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self,x,t):
        y = self.predict(x)
        return self.lastLayer.forward(y,t)

    def gradient(self,x,t):
        self.loss(x,t)
        dout = self.lastLayer.backward(1) # 
        layers_tmp = list(self.layers.values())
        layers_tmp.reverse()
        for layer in layers_tmp:
            dout = layer.backward(dout)

        grads = {}
        grads['W1'], grads['B1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['B2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        return grads
    
    

In [16]:
nn = TwoLayerNeuralNet(input_size=784, hidden_size=50, output_size=10)

In [17]:
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
loss_arr = []
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = nn.gradient(x_batch,t_batch)
    for key in ("W1","B1","W2","B2"):
        nn.params[key] -= learning_rate * grad[key]
    loss = nn.loss(x_batch,t_batch)
    loss_arr.append(loss)


In [26]:
for i in range(10000,20000):
    
    print([int(item) for item in softmax(nn.predict(x_train[i])) == np.max(softmax(nn.predict(x_train[i])))])
    print(t_train[0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0

0.9999999730979775